In [1]:
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR # for building SVR model
import scipy
import catboost as cb
from LocalRegression.bmlr import BMLR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import ray
import pickle
from pyearth import Earth

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def generate_Himmelblau(n=100, sigma=0.1):
    norm = ss.norm(loc=0, scale=sigma)
    x = (ss.uniform.rvs(size=(n, 2))-0.5)*5*2
    y = (x[:, 0]**2 + x[:, 1] - 11)**2 + (x[:, 0] + x[:, 1]**2 - 7)**2
    # scale so that all x points are in [0,1]x[0,1] and all y points are in [0,1]
    x = (x + 5)/10
    y = y/890
    if sigma>0:
        y += norm.rvs(n)
    return x, y

In [4]:
def model_random_forest_params(x, y):
    param_grid = {'n_estimators': [10, 100, 1000, 2500, 3000]}
    rf = RandomForestRegressor()
    gsh = GridSearchCV(rf, param_grid, cv=3).fit(x, y)
    return gsh.best_params_

def model_random_forest(x, y, x_test, y_test, params):
    model = RandomForestRegressor(**params)
    model.fit(x, y)
    pred = model.predict(x_test)
    score = mean_squared_error(y_test, pred, squared=False)
    mape = mean_absolute_percentage_error(y_test, pred)
    return score, mape, pred

def model_catboost(x, y, x_test, y_test):
    train_dataset = cb.Pool(x, y)
    test_dataset = cb.Pool(x_test, y_test)
    model = cb.CatBoostRegressor(loss_function='RMSE', verbose=0)
    model.fit(train_dataset)
    pred = model.predict(test_dataset)
    score = mean_squared_error(y_test, pred, squared=False)
    mape = mean_absolute_percentage_error(y_test, pred)
    return score, mape, pred

def model_mars_params(x, y):
    param_grid = {'max_terms': [5, 10, 100, 200],
                 'max_degree': [1, 2, 3, 4]}
    mars = Earth()
    gsh = GridSearchCV(mars, param_grid, cv=3, verbose=1).fit(x, y)
    return gsh.best_params_

def model_mars(x, y, x_test, y_test, params):
    model = Earth(**params)
    model.fit(x,y)
    pred = model.predict(x_test)
    score = mean_squared_error(y_test, pred, squared=False)
    mape = mean_absolute_percentage_error(y_test, pred)
    return score, mape, pred    

def model_svr_params(x, y):
    param_grid = {'C': [0.1, 1, 100, 1000, 1000],
                  'degree': [2],
                  'epsilon': [0.01, 0.1, 1, 10]
                 }
    svr = SVR(kernel='rbf')
    gsh = GridSearchCV(svr, param_grid, cv=3, verbose=1).fit(x, y)
    return gsh.best_params_

def model_svr(x, y, x_test, y_test, params):
    model = SVR(kernel='rbf', **params)
    model.fit(x, y)
    pred = model.predict(x_test)
    score = mean_squared_error(y_test, pred, squared=False)
    mape = mean_absolute_percentage_error(y_test, pred)
    return score, mape, pred

def model_bmlr_params(x, y, cut, M, include_y):
    epsilon_trial = (np.mean(np.std(x, axis=0)))/np.sqrt(x.shape[0])*3
    param_grid = {'epsilon': [epsilon_trial*x for x in [0.5, 0.8, 1.0, 1.5, 2.0, 2.5, 3.0]]}
    #param_grid = {'epsilon': [epsilon_trial*x for x in [0.5, 0.8]]}
    bmlr = BMLR(cut=cut, M=M, substitution_policy='nearest', include_y=include_y)
    gsh = GridSearchCV(bmlr, param_grid, cv=3, verbose=1).fit(x, y)
    print(gsh.best_params_)
    return gsh.best_params_

def model_bmlr(x, y, x_test, y_test, cut, M, include_y, params):
    model = BMLR(cut=cut, M=M, substitution_policy='nearest', include_y=include_y, **params)
    model.fit(x, y)
    pred = model.predict(x_test)
    score = mean_squared_error(y_test, pred, squared=False)
    mape = mean_absolute_percentage_error(y_test, pred)
    return score, mape, pred

In [5]:
ray.init()

{'node_ip_address': '192.168.1.200',
 'raylet_ip_address': '192.168.1.200',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-04-13_14-18-16_473902_89594/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-04-13_14-18-16_473902_89594/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-04-13_14-18-16_473902_89594',
 'metrics_export_port': 56619,
 'gcs_address': '192.168.1.200:64465',
 'address': '192.168.1.200:64465',
 'node_id': 'de95e2ba35b5beb3ced5fce2d45da454d2e221e0b0542d56690ff231'}

In [6]:
@ray.remote
def run_experiment(ns=[100, 200, 500], mcloops=20, sigma=0.01):
    test_ratio = 0.3
    M=25
    
    scores = []
    for n in ns:
        # find optimal params for each method
        # to speed up parameters are established on one sample only
        sample_x, sample_y = generate_Himmelblau(n=int(n*(1-test_ratio)), sigma=sigma)
        rf_params = model_random_forest_params(sample_x, sample_y)
        svr_params = model_svr_params(sample_x, sample_y)
        mars_params = model_mars_params(sample_x, sample_y)
        # bmlr_params_0 = model_bmlr_params(sample_x, sample_y, cut=0, M=M, include_y=False)
        # bmlr_params_1 = model_bmlr_params(sample_x, sample_y, cut=10, M=M, include_y=False)
        # bmlr_params_2 = model_bmlr_params(sample_x, sample_y, cut=20, M=M, include_y=False)
        # bmlr_params_3 = model_bmlr_params(sample_x, sample_y, cut=30, M=M, include_y=False)
        bmlr_params_0y = model_bmlr_params(sample_x, sample_y, cut=0, M=M, include_y=True)
        bmlr_params_1y = model_bmlr_params(sample_x, sample_y, cut=10, M=M, include_y=True)
        bmlr_params_2y = model_bmlr_params(sample_x, sample_y, cut=20, M=M, include_y=True)
        bmlr_params_3y = model_bmlr_params(sample_x, sample_y, cut=30, M=M, include_y=True)
        
        bmlr_score0, bmlr_mape0 = 0, 0
        bmlr_score1, bmlr_mape1 = 0, 0
        bmlr_score2, bmlr_mape2 = 0, 0
        bmlr_score3, bmlr_mape3 = 0, 0
        
        for mcloop in range(mcloops):
            sample_x, sample_y = generate_Himmelblau(n=n, sigma=sigma)
            sample_x, sample_x_test, sample_y, sample_y_test = train_test_split(sample_x, sample_y, test_size=test_ratio)
            
            rf_score, rf_mape, _ = model_random_forest(sample_x, sample_y, sample_x_test, sample_y_test, rf_params)
            svr_score, svr_mape, _ = model_svr(sample_x, sample_y, sample_x_test, sample_y_test, svr_params)
            mars_score, mars_mape, _ = model_mars(sample_x, sample_y, sample_x_test, sample_y_test, mars_params)
            # bmlr_score0, bmlr_mape0, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=0, M=M, include_y=False, params=bmlr_params_0)
            # bmlr_score1, bmlr_mape1, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=10, M=M, include_y=False, params=bmlr_params_1)
            # bmlr_score2, bmlr_mape2, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=20, M=M, include_y=False, params=bmlr_params_2)
            # bmlr_score3, bmlr_mape3, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=30, M=M, include_y=False, params=bmlr_params_3)
            bmlr_score0y, bmlr_mape0y, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=0, M=M, include_y=True, params=bmlr_params_0y)
            bmlr_score1y, bmlr_mape1y, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=10, M=M, include_y=True, params=bmlr_params_1y)
            bmlr_score2y, bmlr_mape2y, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=20, M=M, include_y=True, params=bmlr_params_2y)
            bmlr_score3y, bmlr_mape3y, _ = model_bmlr(sample_x, sample_y, sample_x_test, sample_y_test, cut=30, M=M, include_y=True, params=bmlr_params_3y)
            cb_score, cb_mape, _ = model_catboost(sample_x, sample_y, sample_x_test, sample_y_test)
            
            scores.append([n, sigma, rf_score, cb_score, svr_score, mars_score, rf_mape, cb_mape, svr_mape, mars_mape,
                           bmlr_score0, bmlr_score1, bmlr_score2, bmlr_score3,
                          bmlr_score0y, bmlr_score1y, bmlr_score2y, bmlr_score3y])
    df = pd.DataFrame(scores, columns=['n', 'sigma', 'rmse_RF', 'rmse_CB', 'rmse_SVR', 'rmse_MARS', 'mape_RF', 'mape_CB', 'mape_SVR', 'mape_MARS',
                                       'rmse_BMLR_cut0', 'rmse_BMLR_cut10', 'rmse_BMLR_cut20', 'rmse_BMLR_cut30',
                                       'rmse_BMLR_cut0_y', 'rmse_BMLR_cut10_y', 'rmse_BMLR_cut20_y', 'rmse_BMLR_cut30_y'])
    return df

In [ ]:
ncpu = 14
loops = 1

for sigma in [0.0, 0.01, 0.1]:
    results = []
    for loop in range(loops):
        print(f'sigma={sigma} loop={loop}')
        res = ray.get([run_experiment.remote(sigma=sigma) for i in range(ncpu)])
        results = results + res
    with open(f'comp_himmelbau_withmars_sigma={sigma}.pickle', "wb") as mypicklefile:
        pickle.dump(results, mypicklefile)

sigma=0.0 loop=0
(run_experiment pid=89732) Fitting 3 folds for each of 20 candidates, totalling 60 fits
(run_experiment pid=89725) Fitting 3 folds for each of 20 candidates, totalling 60 fits
(run_experiment pid=89732) Fitting 3 folds for each of 16 candidates, totalling 48 fits


(run_experiment pid=89732) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89732) To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
(run_experiment pid=89732)   pruning_passer.run()
(run_experiment pid=89732) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89732) To use the future default and silence this warning we advise 

(run_experiment pid=89725) Fitting 3 folds for each of 16 candidates, totalling 48 fits
(run_experiment pid=89730) Fitting 3 folds for each of 20 candidates, totalling 60 fits
(run_experiment pid=89731) Fitting 3 folds for each of 20 candidates, totalling 60 fits
(run_experiment pid=89730) Fitting 3 folds for each of 16 candidates, totalling 48 fits
(run_experiment pid=89731) Fitting 3 folds for each of 16 candidates, totalling 48 fits
(run_experiment pid=89728) Fitting 3 folds for each of 20 candidates, totalling 60 fits


(run_experiment pid=89731) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89731) To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
(run_experiment pid=89731)   pruning_passer.run()
(run_experiment pid=89731) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89731) To use the future default and silence this warning we advise 

(run_experiment pid=89726) Fitting 3 folds for each of 20 candidates, totalling 60 fits
(run_experiment pid=89728) Fitting 3 folds for each of 16 candidates, totalling 48 fits
(run_experiment pid=89726) Fitting 3 folds for each of 16 candidates, totalling 48 fits
(run_experiment pid=89729) Fitting 3 folds for each of 20 candidates, totalling 60 fits
(run_experiment pid=89729) Fitting 3 folds for each of 16 candidates, totalling 48 fits


...

(run_experiment pid=89728) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89728) To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
(run_experiment pid=89728)   pruning_passer.run()
(run_experiment pid=89729) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89729) To use the future default and silence this warning we advise t

(run_experiment pid=89733) {'epsilon': 0.11111071840063325}


(run_experiment pid=89733) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89733) To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
(run_experiment pid=89733)   pruning_passer.run()
(run_experiment pid=89736) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89736) To use the future default and silence this warning we advise t

(run_experiment pid=89730) {'epsilon': 0.10194714627881216}


(run_experiment pid=89730) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89730) To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
(run_experiment pid=89730)   coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]
(run_experiment pid=89725) /home/rafal/LocalRegression/localised-regression/venv/lib/python3.9/site-packages/sklearn_contrib_py_earth-0.1.0+0.gb209d19.dirty-py3.9-linux-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
(run_experiment pid=89725) To use the future default